In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
!ls ../input
# Any results you write to the current directory are saved as output.

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.nn import CosineSimilarity
from torchvision.transforms import ToTensor
from torchvision import transforms
import torchvision.models as models
from pathlib import Path
import PIL.Image
import random
import math
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ExponentialLR
import os
import glob
import cv2
import numpy as np
import PIL
from sklearn.metrics import cohen_kappa_score

In [ ]:
Path('/tmp/.cache/torch/checkpoints/').mkdir(exist_ok=True, parents=True)
!cp /kaggle/input/fastai-pretrained-models/* /tmp/.cache/torch/checkpoints/
!cp /kaggle/input/pretrained-pytorch-models/* /tmp/.cache/torch/checkpoints/
!cp /kaggle/input/resnet152/* /tmp/.cache/torch/checkpoints/
!mv /tmp/.cache/torch/checkpoints/resnet152.pth /tmp/.cache/torch/checkpoints/resnet152-b121ed2d.pth
!ls -lh /tmp/.cache/torch/checkpoints/

In [ ]:
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
path = Path('../input/aptos2019-blindness-detection')

In [ ]:
class CircleCrop:
    def __init__(self, radius):
        self.radius = radius
        circle_img = np.zeros((2 * radius, 2 * radius), np.uint8)
        cv2.circle(circle_img, (radius, radius), int(radius), 1, thickness=-1)
        self.mask = torch.tensor(circle_img, dtype=torch.float32)
        
    def __call__(self, img):
#         assert(img.shape[1] == 2 * self.radius)
#         assert(img.shape[2] == 2 * self.radius)
        return torch.mul(img, self.mask)
    
    def __repr__(self):
        return self.__class__.__name__ + '(radius={0})'.format(self.radius)

In [ ]:
class APTOSDataset(Dataset):
    def __init__(self, path, df, labels=None, augmentation=False, img_size=480):
        self.len = len(df)
        self.path = path
        self.labels = labels
        self.images = list(map(lambda f: path/(f + '.png'), df['id_code']))
        self.trfms = transforms.Compose([
            transforms.Resize(img_size),
            transforms.CenterCrop(img_size),
            transforms.RandomApply((
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.ColorJitter(
                    saturation=(1.0, 1.0),
                    brightness=(1.0, 1.5), 
                    contrast=(1.5, 2.5),
                ),
                transforms.RandomAffine(degrees=180, scale=(1.0, 1.0)),
                ), p = 1 if augmentation else 0  
            ),
            transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225]),
            CircleCrop(img_size // 2),
            ])
        
    def __getitem__(self, index):
        if self.labels is not None:
            label = self.labels[index]
        else:
            label = 0
        path = self.images[index]
        img = PIL.Image.open(path)
        return self.trfms(img), label
    
    def __len__(self):
        return self.len

In [ ]:
tr_df = pd.read_csv(path/'train.csv')

In [ ]:
tr_df.head()

In [ ]:
tr_ds = APTOSDataset(path/'train_images', tr_df, tr_df['diagnosis'].values, augmentation=True, img_size=256)

In [ ]:
te_df = pd.read_csv(path/'test.csv')

In [ ]:
te_ds = APTOSDataset(path/'test_images', te_df, te_df['id_code'], augmentation=True, img_size=256)

In [ ]:
len(tr_ds), len(te_ds)

In [ ]:
img, label = tr_ds[random.randint(0, len(tr_ds) - 1)]
plt.imshow(transforms.ToPILImage()(img))

In [ ]:
img, label = te_ds[random.randint(0, len(te_ds) - 1)]
plt.imshow(transforms.ToPILImage()(img))

In [ ]:
bs = 50
nw = 4

In [ ]:
tr_dl = DataLoader(tr_ds, batch_size=bs, num_workers=nw, drop_last=False, pin_memory=True, shuffle=True)

In [ ]:
te_dl = DataLoader(te_ds, batch_size=bs, num_workers=nw, drop_last=False, pin_memory=True, shuffle=False)

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

In [ ]:
class Bias(nn.Module):
    def __init__(self, num_out):
        super(Bias, self).__init__()
        self.bias = nn.Parameter(torch.zeros(num_out).float())
        
    def forward(self, x):
        return x + self.bias

In [ ]:
num_classes = 5

In [ ]:
model = models.densenet201(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Sequential(
    nn.BatchNorm1d(1920),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=1920, out_features=512),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(512),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=num_classes),
    nn.Linear(in_features=num_classes, out_features=1, bias=False),
    Bias(num_classes - 1)
#     nn.Hardtanh(min_val=0.0, max_val=4.0)
)
# model.fc = nn.Linear(in_features=2048, out_features=num_classes)
# model.classifier.requires_grad = True
model = model.cuda()

In [ ]:
checkpoint = torch.load('../input/aptos-pytorch/checkpoint.tar')
model.load_state_dict(checkpoint['model'])
# optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
tta_count = 1

In [ ]:
from collections import defaultdict
diagnoses = defaultdict(list)
with torch.no_grad():
    for i in tqdm(range(tta_count)):
        id_code = []
        diagnosis = []
        for inputs, id_codes in tqdm(te_dl):
            inputs = inputs.cuda()
#             preds = model(inputs).argmax(1).tolist()
            preds = model(inputs).tolist()
            diagnosis.extend(preds)
            id_code.extend(id_codes)
        for id_c, diag in zip(id_code, diagnosis):
            diagnoses[id_c].append(diag)

In [ ]:
def coral_out_to_class(out):
    out = torch.sigmoid(torch.tensor(out))
#     print(out)
    out = out > 0.5
    return torch.sum(out, dim=1)

In [ ]:
id_code = []
diagnosis = []
for k, v in diagnoses.items():
#     print(k, np.array(v).mean(axis=0), np.array(v).argmax(axis=1), np.array(v))
    id_code.append(k)
#     print(k, v)
#     print(np.bincount(coral_out_to_class(v).numpy()))
#     print(np.argmax(np.bincount(coral_out_to_class(v).numpy())))
#     break
    # mean
#     diagnosis.append((np.array(v).mean(axis=0).argmax()))
#     diagnosis.append(np.minimum(np.array(v).mean(axis=0).round(), 4.0).astype(int)[0])
    # voting
#     diagnosis.append((np.argmax(np.bincount(np.array(v).argmax(axis=1)))))
    diagnosis.append(np.argmax(np.bincount(coral_out_to_class(v).numpy())))

In [ ]:
df = pd.DataFrame({'id_code':id_code, 'diagnosis':diagnosis})
df.to_csv('submission.csv', index=False)